In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc, pickle

/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
df_train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
df_train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

df_test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
df_test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [3]:
df_train = pd.merge(df_train_transaction, df_train_identity, on=['TransactionID'], how='left')
df_test = pd.merge(df_test_transaction, df_test_identity, on=['TransactionID'], how='left')

del df_train_identity, df_train_transaction, df_test_identity, df_test_transaction
gc.collect()

11

In [4]:
def fileter_numeric_cols(data):

    numeric_cols = set()
    numric_cols_cnt = 0

    for col in data.columns:
        col_type = str(data[col].dtypes)
        #print(col_type, type(col_type))
        if col_type[:3] == 'int' or col_type[:5] == 'float' or col_type[:6] == 'double':
            numeric_cols.add(col_type)
            numric_cols_cnt  += 1

    return list(numeric_cols), numric_cols_cnt, numric_cols_cnt/len(data.columns)

train_num_cols, train_numric_cols_cnt, train_numric_cols_rate = fileter_numeric_cols(df_train)

print("数字相关的列有{0}个，类型为{1}个，占比{2}\n{3}".format(train_numric_cols_cnt, len(train_num_cols), \
                                              train_numric_cols_rate, train_num_cols))

数字相关的列有403个，类型为2个，占比0.9285714285714286
['int64', 'float64']


In [5]:
def reduce_mem_usage(df, numerics, verbose=True):
    #numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, \
                                                                    100 * (start_mem - end_mem) / start_mem))
    return df

df_train = reduce_mem_usage(df_train, train_num_cols)
df_test = reduce_mem_usage(df_test, train_num_cols)

Mem. usage decreased to 650.48 Mb (66.8% reduction)
Mem. usage decreased to 565.37 Mb (66.3% reduction)


In [6]:
with open('df_merged_train.pkl', 'wb') as f:
    pickle.dump(df_train, f)
    print("df_train dump finished")
    
with open('df_merged_test.pkl', 'wb') as f:
    pickle.dump(df_test, f)    
    print("df_test dump finished")

df_train dump finished
df_test dump finished


In [7]:
SPLIT_POINT = 590540

df_all = pd.concat((df_train, df_test), axis=0).reset_index()

del df_train, df_test

gc.collect()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


0

In [8]:
del_all = False

if del_all:

    data_null = df_all.isnull().sum()
    data_null = data_null.drop(data_null[data_null == 0].index).sort_values(ascending=False)

    data_null = data_null / df_all.shape[0] * 100

    print("非空列有{0}个 空列占比{1}%".format(df_all.shape[1] - len(data_null), len(data_null) / df_all.shape[1] * 100))

else:
    
    need_drop = ['V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
    
    df_all = df_all.drop(need_drop, axis=1)

In [9]:
null_rate = 0.9 * 100

if del_all:

    plt.plot(np.arange(data_null.shape[0]), data_null)
    plt.plot(np.arange(data_null.shape[0]), np.array([null_rate] * data_null.shape[0]))

    print("null rate >= {0} 的列有{1}个，占比总null列{2:.5f}%".format(null_rate, np.sum(data_null > null_rate), \
                                                         np.sum(data_null > null_rate) / data_null.shape[0]  * 100 ))

In [10]:
if del_all:
    #担心identity融入后，导致identity的特征在all中空值占比太高
    df_all = df_all.drop(data_null[data_null > null_rate].index, axis=1)


In [11]:
def fill_numbers(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")

    for col in props.columns:
        if props[col].dtype != object:  
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all():  
                props[col].fillna(mn-1,inplace=True)  

    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

df_all = fill_numbers(df_all)

Memory usage of properties dataframe is : 1056.865156173706  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1056.865156173706  MB
This is  100.0 % of the initial size


In [12]:
data_null_catogory = df_all.isnull().sum()
data_null_catogory = data_null_catogory.drop(data_null_catogory[data_null_catogory == 0].index)   \
                        .sort_values(ascending=False)

data_null_catogory = data_null_catogory / df_all.shape[0] * 100

In [13]:
data_null_catogory

id_27            99.067562
id_23            99.067562
id_33            86.879700
id_30            86.491085
id_34            86.331046
DeviceInfo       78.698834
id_16            76.751751
R_emaildomain    75.104513
id_31            74.763108
DeviceType       74.687099
id_29            74.685732
id_28            74.685732
id_37            74.666957
id_38            74.666957
id_15            74.666957
id_35            74.666957
id_36            74.666957
id_12            73.921626
M5               60.161807
M7               52.977267
M9               52.974807
M8               52.974807
M4               47.318113
M3               40.806266
M2               40.806266
M1               40.806266
M6               29.920682
P_emaildomain    14.914635
card4             0.424979
card6             0.417232
dtype: float64

In [14]:
for col in data_null_catogory.index:
    print("Columns: ", col)
    print(df_all[col].value_counts().shape)

Columns:  id_27
(2,)
Columns:  id_23
(3,)
Columns:  id_33
(461,)
Columns:  id_30
(87,)
Columns:  id_34
(4,)
Columns:  DeviceInfo
(2799,)
Columns:  id_16
(2,)
Columns:  R_emaildomain
(60,)
Columns:  id_31
(172,)
Columns:  DeviceType
(2,)
Columns:  id_29
(2,)
Columns:  id_28
(2,)
Columns:  id_37
(2,)
Columns:  id_38
(2,)
Columns:  id_15
(3,)
Columns:  id_35
(2,)
Columns:  id_36
(2,)
Columns:  id_12
(2,)
Columns:  M5
(2,)
Columns:  M7
(2,)
Columns:  M9
(2,)
Columns:  M8
(2,)
Columns:  M4
(3,)
Columns:  M3
(2,)
Columns:  M2
(2,)
Columns:  M1
(2,)
Columns:  M6
(2,)
Columns:  P_emaildomain
(60,)
Columns:  card4
(4,)
Columns:  card6
(4,)


In [15]:
df_all['DeviceInfo'].value_counts().head(10)

Windows                  92710
iOS Device               38502
MacOS                    23722
Trident/7.0              12330
rv:11.0                   2650
SM-G532M Build/MMB29T      980
rv:57.0                    968
SM-J700M Build/MMB29K      829
SM-G610M Build/MMB29K      804
SM-G531H Build/LMY48B      682
Name: DeviceInfo, dtype: int64

In [16]:
df_all['id_33'].value_counts().head(10)

1920x1080    33742
1366x768     15046
1334x750     11550
2208x1242     9113
1440x900      8127
1600x900      6603
2048x1536     6556
2436x1125     4230
2560x1600     3871
1280x800      3704
Name: id_33, dtype: int64

In [17]:
df_all['id_30'].value_counts().head(10)

Windows 10          42170
Windows 7           23478
iOS 12.1.0           6349
Mac OS X 10_12_6     3884
iOS 11.2.1           3824
Mac OS X 10_11_6     3802
iOS 11.1.2           3776
Android 7.0          3573
iOS 11.4.1           3539
Windows 8.1          3308
Name: id_30, dtype: int64

In [18]:
tmp = df_all.copy()

In [19]:
means_email  = True

if means_email:
    #add new feature

    #email
    yahho_email = ["yahoo.fr", "yahoo.de", "yahoo.es", "yahoo.co.uk", "yahoo.com", "yahoo.com.mx", "ymail.com", "rocketmail.com", "frontiernet.net"]
    df_all['P_isyahoo'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isyahoo'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)

    micro_email = ["hotmail.com", "live.com.mx", "live.com", "msn.com", "hotmail.es", "outlook.es", "hotmail.fr", "hotmail.de", "hotmail.co.uk"]
    df_all['P_ismicro'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_ismicro'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)

    apple_email = ["icloud.com", "mac.com", "me.com"]
    df_all['P_isapple'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isapple'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)

    att = ["prodigy.net.mx", "att.net", "sbxglobal.net"]
    df_all['P_isprodigy'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isprodigy'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    century_email = ["centurylink.net", "embarqmail.com", "q.com"]
    df_all['P_iscentury'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_iscentury'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    aol_email = ["aim.com", "aol.com"]
    df_all['P_isaol'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isaol'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    spect_email = ["twc.com", "charter.com"]
    df_all['P_isspect'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isspect'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    proton_email = ["protonmail.com"]
    df_all['P_isproton'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isproton'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    comcast_email = ["comcast.net"]
    df_all['P_iscomcast'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_iscomcast'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    gg_email = ["gmail.com"]
    df_all['P_isgg'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isgg'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    anoy_email = ["anonymous.com"]
    df_all['P_isanoy'] = df_all['P_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    df_all['R_isanoy'] = df_all['R_emaildomain'].isin(yahho_email).apply(lambda x:1 if x else 0)
    
    df_all['P_isNan'] = df_all['P_emaildomain'].isnull().apply(lambda x:1 if x else 0)
    df_all['R_isNan'] = df_all['R_emaildomain'].isnull().apply(lambda x:1 if x else 0)
    
    df_all.drop(['P_emaildomain', 'R_emaildomain'], inplace=True, axis=1)

In [20]:
new_browsers  = ["samsung browser 7.0", "opera 53.0", "mobile safari 10.0", "google search application 49.0",
                 "firefox 60.0", "edge 17.0", "chrome 69.0", "chrome 67.0", "chrome 63.0", "chrome 63.0", 
                 "chrome 64.0", "chrome 64.0 for android", "chrome 64.0 for ios", "chrome 65.0", "chrome 65.0 for android",
                 "chrome 65.0 for ios", "chrome 66.0", "chrome 66.0 for android", "chrome 66.0 for ios"]

df_all['is_new_browser'] = df_all['id_31'].isin(new_browsers).apply(lambda x:1 if x else 0)
df_all.drop(['id_31'], inplace=True, axis=1)

In [21]:
if not means_email:
    #应该还可以提高，不然others类别占比比正常值还高
    use_top_email = 8

    use_top_email_cols = (df_all['P_emaildomain'].value_counts(dropna=False) / \
                          df_all.shape[0] * 100)[:use_top_email]

    print('top {0} email domain rank {1} %'.format(use_top_email, use_top_email_cols.sum()))

    df_all['P_emaildomain'] = df_all['P_emaildomain'].apply(lambda x: \
                                                            x if x in use_top_email_cols.index else 'others')

In [22]:
if not means_email:
    use_top_email_cols


In [23]:
'''
Windows                  92710
iOS Device               38502
MacOS                    23722
Trident/7.0              12330
'''
#后期应该统计下全部，相似名称的放一起


df_all['device_is_win'] = df_all['DeviceInfo'].isin(["Windows"]).apply(lambda x:1 if x else 0)
df_all['device_is_ios'] = df_all['DeviceInfo'].isin(["iOS Device "]).apply(lambda x:1 if x else 0)
df_all['device_is_mac'] = df_all['DeviceInfo'].isin(["MacOS"]).apply(lambda x:1 if x else 0)
df_all['device_is_thrid'] = df_all['DeviceInfo'].isin(["Trident/7.0"]).apply(lambda x:1 if x else 0)
df_all['device_is_nan'] = df_all['DeviceInfo'].isnull().apply(lambda x:1 if x else 0)

df_all.drop(['DeviceInfo'], axis=1, inplace=True)

In [24]:
'''
1920x1080    33742
1366x768     15046
1334x750     11550
'''

w_size = ["1920x1080", "1366x768", "1334x750"]

df_all['id_33'] = df_all['id_33'].isin(["Windows"]).apply(lambda x:x if x else "others")

In [25]:
'''
Windows 10          42170
Windows 7           23478
iOS 12.1.0           6349
'''

w_sys = ["Windows 10", "Windows 7", "iOS 12.1.0"]

df_all['id_30'] = df_all['id_30'].isin(["Windows"]).apply(lambda x:x if x else "others")

In [26]:
#手动删点高取值范围的特征

need_drop_catogory = []

tmp = df_all.copy()
df_all.drop(need_drop_catogory, axis=1, inplace=True)

In [27]:
#d1 = pd.get_dummies(df_train['M1'], dummy_na=True, prefix='M1_')
#except P_emaildomain
#部分列取值范围不一样，甚至test取值比train取值更广，是以train的为准，还是全集为准
def fill_dummy_category(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    print("before, data shape {0}".format(props.shape))
    need_drop = []
    need_add = pd.DataFrame()
    for col in props.columns:
        if props[col].dtype == object:  # Exclude strings
            
            need_drop.append(col)
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            print("取值种类: {0}".format(len(props[col].value_counts(dropna=False).index)))
            
            dummy = pd.get_dummies(props[col], dummy_na=True, prefix='%s_'%col)
            
            need_add = pd.concat((need_add, dummy), axis=1)

            #处理完一列，删一列
            props = props.drop(col, axis=1)
    props = pd.concat((props, need_add), axis=1)
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    print("after, data shape {0}".format(props.shape))
    return props


df_all_dy = fill_dummy_category(df_all)

del df_all
gc.collect()

Memory usage of properties dataframe is : 1274.5165691375732  MB
before, data shape (1097231, 387)
******************************
Column:  DeviceType
取值种类: 3
******************************
Column:  M1
取值种类: 3
******************************
Column:  M2
取值种类: 3
******************************
Column:  M3
取值种类: 3
******************************
Column:  M4
取值种类: 4
******************************
Column:  M5
取值种类: 3
******************************
Column:  M6
取值种类: 3
******************************
Column:  M7
取值种类: 3
******************************
Column:  M8
取值种类: 3
******************************
Column:  M9
取值种类: 3
******************************
Column:  ProductCD
取值种类: 5
******************************
Column:  card4
取值种类: 5
******************************
Column:  card6
取值种类: 5
******************************
Column:  id_12
取值种类: 3
******************************
Column:  id_15
取值种类: 4
******************************
Column:  id_16
取值种类: 3
******************************
Column:  id_23
取值种类: 4
*

0

In [28]:
#显示所有列
pd.set_option('display.max_columns', None)


df_all_dy.head()

,index,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D7,D8,D9,TransactionAmt,TransactionDT,TransactionID,V10,V100,V101,V103,V11,V12,V126,V127,V128,V129,V13,V130,V131,V132,V139,V140,V141,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V197,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V285,V287,V288,V290,V291,V292,V294,V296,V298,V3,V30,V306,V307,V308,V31,V310,V312,V313,V314,V317,V32,V322,V323,V324,V326,V329,V33,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V66,V67,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,addr1,addr2,card1,card2,card3,card5,dist1,dist2,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32,isFraud,P_isyahoo,R_isyahoo,P_ismicro,R_ismicro,P_isapple,R_isapple,P_isprodigy,R_isprodigy,P_iscentury,R_iscentury,P_isaol,R_isaol,P_isspect,R_isspect,P_isproton,R_isproton,P_iscomcast,R_iscomcast,P_isgg,R_isgg,P_isanoy,R_isanoy,P_isNan,R_isNan,is_new_browser,device_is_win,device_is_ios,device_is_mac,device_is_thrid,device_is_nan,DeviceType__desktop,DeviceType__mobile,DeviceType__nan,M1__F,M1__T,M1__nan,M2__F,M2__T,M2__nan,M3__F,M3__T,M3__nan,M4__M0,M4__M1,M4__M2,M4__nan,M5__F,M5__T,M5__nan,M6__F,M6__T,M6__nan,M7__F,M7__T,M7__nan,M8__F,M8__T,M8__nan,M9__F,M9__T,M9__nan,ProductCD__C,ProductCD__H,ProductCD__R,ProductCD__S,ProductCD__W,ProductCD__nan,card4__american express,card4__discover,card4__mastercard,card4__visa,card4__nan,card6__charge card,card6__credit,card6__debit,card6__debit or credit,card6__nan,id_12__Found,id_12__NotFound,id_12__nan,id_15__Found,id_15__New,id_15__Unknown,id_15__nan,id_16__Found,id_16__NotFound,id_16__nan,id_23__IP_PROXY:ANONYMOUS,id_23__IP_PROXY:HIDDEN,id_23__IP_PROXY:TRANSPARENT,id_23__nan,id_27__Found,id_27__NotFound,id_27__nan,id_28__Found,id_28__New,id_28__nan,id_29__Found,id_29__NotFound,id_29__nan,id_30__True,id_30__others,id_30__nan,id_33__others,id_33__nan,id_34__match_status:-1,id_34__match_status:0,id_34__match_status:1,id_34__match_status:2,id_34__nan,id_35__F,id_35__T,id_35__nan,id_36__F,id_36__T,id_36__nan,id_37__F,id_37__T,id_37__nan,id_38__F,id_38__T,id_38__nan
0,0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,14.0,13.0,13.0,-84.0,-1.0,-194.0,0.0,-1.0,13.0,-123.0,-1.0,-84.0,-1.0,-1.0,-1.0,68.5,86400,2987000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.

In [29]:
def handle_dt(data):
    
    data['TransactionDT'] = data['TransactionDT'].apply(lambda x:(x+59) // 60 // 60 % 24)
    
    '''
    t1 = 3.421434e+07
    t2 = 8.640000e+04

    t1-t2, 24*60*60
    
    #out 
    (34127940.0, 86400)
    '''
    
    def judge_zone(dt_hour):
        #zone = [(7, 12), (12, 17), (17, 23)]# other is (23, 7)
        zone = [(4, 11)]
        #ch = 3
        for idx, zn in enumerate(zone):
            if dt_hour >= zn[0] and dt_hour < zn[1]:
                return idx
        return 3
    
    data['TransactionDT'] = data['TransactionDT'].apply(lambda x:judge_zone(x))
    #print(data['TransactionDT'].describe())
    
    dummy = pd.get_dummies(data['TransactionDT'], dummy_na=False, prefix='%s_' % 'TransactionDT')
    data.drop(['TransactionDT'], axis=1, inplace=True)
    data = pd.concat((data, dummy), axis=1)
    
    return data

#tmp = handle_dt(df_all_dy.copy())
df_all_dy = handle_dt(df_all_dy)

In [30]:
'''
tmp['TransactionDT__0'].sum(), \
tmp['TransactionDT__1'].sum(), \
tmp['TransactionDT__2'].sum(), \
tmp['TransactionDT__3'].sum()

tmp[tmp['isFraud'] == 0][]
'''

"\ntmp['TransactionDT__0'].sum(), tmp['TransactionDT__1'].sum(), tmp['TransactionDT__2'].sum(), tmp['TransactionDT__3'].sum()\n\ntmp[tmp['isFraud'] == 0][]\n"

In [31]:
df_all_dy.head(3)

,index,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D7,D8,D9,TransactionAmt,TransactionID,V10,V100,V101,V103,V11,V12,V126,V127,V128,V129,V13,V130,V131,V132,V139,V140,V141,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V197,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V285,V287,V288,V290,V291,V292,V294,V296,V298,V3,V30,V306,V307,V308,V31,V310,V312,V313,V314,V317,V32,V322,V323,V324,V326,V329,V33,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V66,V67,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,addr1,addr2,card1,card2,card3,card5,dist1,dist2,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32,isFraud,P_isyahoo,R_isyahoo,P_ismicro,R_ismicro,P_isapple,R_isapple,P_isprodigy,R_isprodigy,P_iscentury,R_iscentury,P_isaol,R_isaol,P_isspect,R_isspect,P_isproton,R_isproton,P_iscomcast,R_iscomcast,P_isgg,R_isgg,P_isanoy,R_isanoy,P_isNan,R_isNan,is_new_browser,device_is_win,device_is_ios,device_is_mac,device_is_thrid,device_is_nan,DeviceType__desktop,DeviceType__mobile,DeviceType__nan,M1__F,M1__T,M1__nan,M2__F,M2__T,M2__nan,M3__F,M3__T,M3__nan,M4__M0,M4__M1,M4__M2,M4__nan,M5__F,M5__T,M5__nan,M6__F,M6__T,M6__nan,M7__F,M7__T,M7__nan,M8__F,M8__T,M8__nan,M9__F,M9__T,M9__nan,ProductCD__C,ProductCD__H,ProductCD__R,ProductCD__S,ProductCD__W,ProductCD__nan,card4__american express,card4__discover,card4__mastercard,card4__visa,card4__nan,card6__charge card,card6__credit,card6__debit,card6__debit or credit,card6__nan,id_12__Found,id_12__NotFound,id_12__nan,id_15__Found,id_15__New,id_15__Unknown,id_15__nan,id_16__Found,id_16__NotFound,id_16__nan,id_23__IP_PROXY:ANONYMOUS,id_23__IP_PROXY:HIDDEN,id_23__IP_PROXY:TRANSPARENT,id_23__nan,id_27__Found,id_27__NotFound,id_27__nan,id_28__Found,id_28__New,id_28__nan,id_29__Found,id_29__NotFound,id_29__nan,id_30__True,id_30__others,id_30__nan,id_33__others,id_33__nan,id_34__match_status:-1,id_34__match_status:0,id_34__match_status:1,id_34__match_status:2,id_34__nan,id_35__F,id_35__T,id_35__nan,id_36__F,id_36__T,id_36__nan,id_37__F,id_37__T,id_37__nan,id_38__F,id_38__T,id_38__nan,TransactionDT__0,TransactionDT__3
0,0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,14.0,13.0,13.0,-84.0,-1.0,-194.0,0.0,-1.0,13.0,-123.0,-1.0,-84.0,-1.0,-1.0,-1.0,68.5,2987000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0

In [32]:
SPLIT_POINT

590540

In [33]:
#del df_all_dy

#gc.collect()

df_train = df_all_dy[:SPLIT_POINT]
df_test  = df_all_dy[SPLIT_POINT:]

In [34]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

%time clf.fit(df_train.drop(['TransactionID', 'index', 'isFraud'], axis=1), df_train['isFraud'])

CPU times: user 16.5 s, sys: 34.1 s, total: 50.6 s
Wall time: 50.8 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [35]:
sample_submission = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv')
sample_submission['isFraud'] = clf.predict_proba(df_test.drop(['TransactionID', 'index', 'isFraud'], axis=1))[:,1]
sample_submission.to_csv('simple_xgboost-scaler-9-20-k.csv', index=False, columns=['TransactionID', 'isFraud'])

In [36]:
sample_submission.head(10)

,TransactionID,isFraud
0,3663549,0.004922
1,3663550,0.001354
2,3663551,0.002069
3,3663552,0.001029
4,3663553,0.002716
5,3663554,0.003588
6,3663555,0.010754
7,3663556,0.027256
8,3663557,0.000125
9,3663558,0.005922


In [37]:
sample_submission.tail(10)

,TransactionID,isFraud
506681,4170230,0.016981
506682,4170231,0.025969
506683,4170232,0.001485
506684,4170233,0.006307
506685,4170234,0.003970
506686,4170235,0.012580
506687,4170236,0.009845
506688,4170237,0.007735
506689,4170238,0.017168
506690,4170239,0.004744
